In [89]:
import pandas as pd
import numpy as np
from faker import Faker
import pytest
from datetime import timedelta, datetime

fake = Faker()

In [90]:
contract_periods = pd.DataFrame({
        "contract_period_id": range(1, 4),
        "period": [12, 24, 36]
    })

details_rows = []
n = 5
for i in range(n + 1):
    created_at = fake.date_time_between(start_date="-3y", end_date="now")
    last_updated = created_at + timedelta(days=np.random.randint(1, 500))
    
    details_rows.append({
        "contract_details_id": i,
        "contract_title": fake.catch_phrase(),
        "initial_price": round(np.random.uniform(10, 200), 2),
        "discount_percent": round(np.random.choice([0, 5, 10, 15, 20]), 1),
        "contract_period_id": np.random.choice(contract_periods["contract_period_id"]),
        "num_of_sims": np.random.randint(1, 5),
        "num_of_devices": np.random.randint(0, 3),
        "personal_data_id": np.random.randint(1, 50),
        "created_at": created_at,
        "last_updated": last_updated
    })

contract_details = pd.DataFrame(details_rows)

contracts_rows = []
for i in range(n + 1):
    created_at = contract_details.iloc[i]["created_at"]
    last_updated = contract_details.iloc[i]["last_updated"]
    
    contracts_rows.append({
        "contract_id": i,
        "contract_details_id": contract_details.iloc[i]["contract_details_id"],
        "contract_type_id": np.random.choice([1,2,3]),
        "created_at": created_at,
        "last_updated": last_updated
    })

contracts = pd.DataFrame(contracts_rows)


customers_contracts_rows = []
for i in range(n + 1):
    customers_contracts_rows.append({
        "customer_contract_id": i,
        "customer_id": np.random.randint(1, 50),
        "contract_id": contracts.iloc[i]["contract_id"]
    })

customers_contracts = pd.DataFrame(customers_contracts_rows)

In [91]:
contract_details = contract_details.merge(contract_periods,on="contract_period_id",how="left" )

contract_details = contract_details[["contract_details_id", "period"]]
contract_details

,contract_details_id,period
0,0,12
1,1,36
2,2,12
3,3,24
4,4,24
5,5,24


In [92]:
contracts = contracts.merge(contract_details,on="contract_details_id",how="left")

contracts = contracts[["contract_id","created_at", "last_updated","period"]]

contracts

,contract_id,created_at,last_updated,period
0,0,2023-07-29 15:31:38.224526,2023-09-18 15:31:38.224526,12
1,1,2025-04-08 14:13:37.832287,2025-10-23 14:13:37.832287,36
2,2,2022-11-11 03:33:00.076486,2022-12-12 03:33:00.076486,12
3,3,2022-09-01 14:55:03.279831,2023-02-10 14:55:03.279831,24
4,4,2024-10-19 01:00:10.648795,2025-02-10 01:00:10.648795,24
5,5,2023-08-11 20:20:52.836498,2023-11-13 20:20:52.836498,24


In [93]:
customers_contracts = customers_contracts.merge(contracts, on="contract_id",how="left")


customers_contracts["created_at"] = customers_contracts["created_at"].dt.date
customers_contracts["last_updated"] = customers_contracts["last_updated"].dt.date

customers_contracts["start_date"] = customers_contracts["created_at"]
customers_contracts["end_date"] =  customers_contracts.apply(
    lambda row: row["start_date"] + pd.DateOffset(months=row["period"]),
    axis=1
)



customers_contracts["is_active"] = customers_contracts["end_date"] > pd.Timestamp.now()

customers_contracts.drop("customer_contract_id", axis=1, inplace=True)


customers_contracts

,customer_id,contract_id,created_at,last_updated,period,start_date,end_date,is_active
0,32,0,2023-07-29,2023-09-18,12,2023-07-29,2024-07-29,False
1,22,1,2025-04-08,2025-10-23,36,2025-04-08,2028-04-08,True
2,30,2,2022-11-11,2022-12-12,12,2022-11-11,2023-11-11,False
3,12,3,2022-09-01,2023-02-10,24,2022-09-01,2024-09-01,False
4,9,4,2024-10-19,2025-02-10,24,2024-10-19,2026-10-19,True
5,31,5,2023-08-11,2023-11-13,24,2023-08-11,2025-08-11,False
